# Workflow
``chainladder`` facilitates practical reserving workflows.

## Pipeline
The :class:`Pipeline` class implements utilities to build a composite
estimator, as a chain of transforms and estimators.  Said differently, a
`Pipeline` is a way to wrap multiple estimators into a single compact object.
The `Pipeline` is borrowed from scikit-learn.  As an example of compactness,
we can simulate a set of triangles using bootstrap sampling, apply volume-weigted
development, exponential tail curve fitting, and get the 95%-ile IBNR estimate.


In [4]:
import chainladder as cl
import numpy as np

steps=[
    ('sample', cl.BootstrapODPSample(random_state=42)),
    ('dev', cl.Development(average='volume')),
    ('tail', cl.TailCurve('exponential')),
    ('model', cl.Chainladder())]
pipe = cl.Pipeline(steps=steps)
pipe.fit(cl.load_sample('genins'))
pipe.named_steps.model.ibnr_.sum('origin').quantile(q=.95)

26063265.939845018

Each estimator contained within a pipelines ``steps`` can be accessed by name
using the ``named_steps`` attribute of the `Pipeline`.

## VotingChainladder

The :class:`VotingChainladder` ensemble method allows the actuary to vote between
different underlying ``ibnr_`` by way of a matrix of weights.

For example, the actuary may choose 
* the :class:`Chainladder` method for the first 4 origin periods, 
* the :class:`BornhuetterFerguson` method for the next 3 origin periods,
* and the :class:`CapeCod` method for the final 3.

In [5]:
raa = cl.load_sample('RAA')
cl_ult = cl.Chainladder().fit(raa).ultimate_ # Chainladder Ultimate
apriori = cl_ult*0+(cl_ult.sum()/10) # Mean Chainladder Ultimate

bcl = cl.Chainladder()
bf = cl.BornhuetterFerguson()
cc = cl.CapeCod()

estimators = [('bcl', bcl), ('bf', bf), ('cc', cc)]
weights = np.array([[1, 0, 0]] * 4 + [[0, 1, 0]] * 3 + [[0, 0, 1]] * 3)
vot = cl.VotingChainladder(estimators=estimators, weights=weights)
vot.fit(raa, sample_weight=apriori)
vot.ultimate_

,2261
1981,"18,834"
1982,"16,858"
1983,"24,083"
1984,"28,703"
1985,"28,204"
1986,"19,840"
1987,"18,840"
1988,"23,107"
1989,"20,005"
1990,"21,606"


Alternatively, the actuary may choose to combine all methods using weights. Omitting
the weights parameter results in the average of all predictions assuming a weight of 1.
Alternatively, a default weight can be enforced through the `default_weight` parameter.


In [6]:
raa = cl.load_sample('RAA')
cl_ult = cl.Chainladder().fit(raa).ultimate_ # Chainladder Ultimate
apriori = cl_ult * 0 + (float(cl_ult.sum()) / 10) # Mean Chainladder Ultimate

bcl = cl.Chainladder()
bf = cl.BornhuetterFerguson()
cc = cl.CapeCod()

estimators = [('bcl', bcl), ('bf', bf), ('cc', cc)]
vot = cl.VotingChainladder(estimators=estimators)
vot.fit(raa, sample_weight=apriori)
vot.ultimate_

,2261
1981,"18,834"
1982,"16,887"
1983,"24,042"
1984,"28,436"
1985,"28,467"
1986,"19,771"
1987,"18,548"
1988,"23,305"
1989,"18,530"
1990,"20,331"


The weights can take the form of an array (as above), a dict, or a callable:


In [7]:
callable_weight = lambda origin : np.where(origin.year < 1985, (1, 0, 0), np.where(origin.year > 1987, (0, 0, 1), (0, 1, 0)))

dict_weight = {
    '1992': (0, 1, 0),
    '1993': (0, 1, 0),
    '1994': (0, 1, 0),
    '1995': (0, 0, 1),
    '1996': (0, 0, 1),
    '1997': (0, 0, 1),
}  # Unmapped origins will get `default_weight`


GridSearch
==========
The grid search provided by :class:`GridSearch` exhaustively generates
candidates from a grid of parameter values specified with the ``param_grid``
parameter.  Like `Pipeline`, `GridSearch` borrows from its scikit-learn counterpart
``GridSearchCV``.

Because reserving techniques are different from supervised machine learning,
`GridSearch` does not try to pick optimal hyperparameters for you. It is more of
a scenario-testing estimator.

`GridSearch` can be applied to all other estimators, including the `Pipeline`
estimator.  To use it, one must specify a ``param_grid`` as well as a ``scoring``
function which defines the estimator property(s) you wish to capture.  If capturing
multiple properties is desired, multiple scoring functions can be created and
stored in a dictionary.

Here we capture multiple properties of the `TailBondy` estimator using the
`GridSearch` routine to test the sensitivity of the model to changing hyperparameters.

```{eval-rst}
.. figure:: /auto_examples/images/sphx_glr_plot_bondy_sensitivity_001.png
   :target: ../auto_examples/plot_bondy_sensitivity.html
   :align: center
   :scale: 70%
```

Using `GridSearch` for scenario testing is entirely optional.  You can write
your own looping mechanisms to achieve the same result.  For example:

```{eval-rst}
.. figure:: /auto_examples/images/sphx_glr_plot_capecod_001.png
   :target: ../auto_examples/plot_capecod.html
   :align: center
   :scale: 50%
```